In [1]:
import pandas as pd
import dask.dataframe as dd
import glob
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from itertools import groupby
#from heatwave import detect
from datetime import timedelta
import seaborn as sns
from datetime import date
import matplotlib.pyplot as plt
import seaborn
from shapely.geometry import Point

## Process Monthly Collected Data (Washington, Oregon, California)

In [ ]:
full_daily_max = pd.DataFrame()
full_df = pd.DataFrame()



for state in ['California', 'Oregon', 'Washington']:
    

    for year in ['2016', '2017', '2018', '2019', '2020', '2021', '2022']:
        
        data_dic = {}

        monthly_folders = glob.glob(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/temp_data/{state}/{year}/*')

        for folder in monthly_folders:
            
            files = glob.glob(f'{folder}/*')
            print(files)
                
            month_data = dd.read_csv(files)
            month_data = month_data.compute()

            data_dic[folder] = month_data

        temp = pd.concat(data_dic.values()) 

        temp['cell'] = temp['longitude'].astype(str) + ', ' + temp['latitude'].astype(str)

        temp['date'] = pd.to_datetime(temp['datetime']).dt.tz_localize('UTC').dt.tz_convert('America/Los_Angeles')
        temp['day'] = temp['date'].dt.day
        temp['month'] = temp['date'].dt.month
        temp['year'] = year

        #get each counties cell max temperature

        daily_max = temp.groupby(['county', 'cell', 'day', 'month', 'year'])['temperature_2m'].max().reset_index()            

        #get mean maximum

        daily_max = daily_max.groupby(['county', 'day', 'month', 'year'])['temperature_2m'].mean().reset_index()

        daily_max['temperature_2m'] = daily_max['temperature_2m'] - 273.15

        daily_max['state'] = state


        full_daily_max = pd.concat([full_daily_max, daily_max])


        daily_max.to_csv(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/daily_max/{state}/max_monthly_{year}.csv')


    full_df = pd.DataFrame()


## Process Weekly Collected Data (British Columbia, Washington, Orgon, California)

In [ ]:
full_df = pd.DataFrame()
full_daily_max = pd.DataFrame()
for state in ['British Columbia', 'California', 'Oregon', 'Washington']:
    
    for year in ['2016', '2017', '2018', '2019', '2020', '2021', '2022']:
        
        data_dic = {}

        weekly_folders = glob.glob(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/temp_data/{state}/large/{year}/*')

        for folder in weekly_folders:
            
            files = glob.glob(f'{folder}/*')
            print(files)
                
            week_data = dd.read_csv(files)
            week_data = week_data.compute()

            data_dic[folder] = week_data
            
            
        temp = pd.concat(data_dic.values()) 

        temp['cell'] = temp['longitude'].astype(str) + ', ' + temp['latitude'].astype(str)

        temp['date'] = pd.to_datetime(temp['datetime']).dt.tz_localize('UTC').dt.tz_convert('America/Los_Angeles')
        temp['day'] = temp['date'].dt.day
        temp['month'] = temp['date'].dt.month
        temp['year'] = year


        #get each counties cell max temperature

        daily_max = temp.groupby(['county', 'cell', 'day', 'month', 'year'])['temperature_2m'].max().reset_index()            

        #get mean maximum

        daily_max = daily_max.groupby(['county', 'day', 'month', 'year'])['temperature_2m'].mean().reset_index()

        daily_max['temperature_2m'] = daily_max['temperature_2m'] - 273.15

        daily_max['state'] = state

        
        full_daily_max = pd.concat([full_daily_max, daily_max])        
           

        daily_max.to_csv(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/daily_max/{state}/max_weekly_{year}.csv')
        
        full_df = pd.DataFrame()






## Process Daily Collected Data (BC)

In [ ]:
full_df = pd.DataFrame()
full_daily_max = pd.DataFrame()
data_dic = {}
for state in ['British Columbia']:
    
    for year in ['2016', '2017', '2018', '2019', '2020', '2021', '2022']:

        daily_folders = glob.glob(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/temp_data/{state}/final/{year}/*')

        for folder in daily_folders:
            
            files = glob.glob(f'{folder}/*')
            print(files)
                
            daily_data = dd.read_csv(files)
            daily_data = daily_data.compute()
            
            data_dic[folder] = daily_data

        full_df = pd.concat(data_dic.values()) 
        
        
        data_dic = {}
        
        
        temp = full_df.copy()

        temp['cell'] =  temp['longitude'].astype(str) + ', ' + temp['latitude'].astype(str)

        temp['date'] = pd.to_datetime(temp['datetime']).dt.tz_localize('UTC').dt.tz_convert('America/Los_Angeles')
        temp['day'] = temp['date'].dt.day
        temp['month'] = temp['date'].dt.month
        temp['year'] = year


        #get each counties cell max temperature

        daily_max = temp.groupby(['county', 'cell', 'day', 'month', 'year'])['temperature_2m'].max().reset_index()            

        #get mean maximum

        daily_max = daily_max.groupby(['county', 'day', 'month', 'year'])['temperature_2m'].mean().reset_index()

        daily_max['temperature_2m'] = daily_max['temperature_2m'] - 273.15

        daily_max['state'] = state

        
        full_daily_max = pd.concat([full_daily_max, daily_max])        
           

        daily_max.to_csv(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/daily_max/{state}/max_daily_{year}.csv')
        
        full_df = pd.DataFrame()






In [12]:
daily_max_dic = {}

for state in ['California', 'Oregon', 'Washington', 'British Columbia']:
            
    daily_max_files = glob.glob(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/daily_max/{state}/max*')
    
    temp = dd.read_csv(daily_max_files)
    temp = temp.compute()
    daily_max_dic[state] = temp
    

full_df = pd.concat(daily_max_dic.values())

In [15]:
full_df['datetime'] = pd.to_datetime(full_df[['day', 'month', 'year']])
full_df['datetime_ord'] = [date(x,y,z).toordinal() for x,y,z in zip(full_df['year'], full_df['month'], full_df['day'])]

In [16]:
full_df['location'] = full_df['county'] + ', ' + full_df['state']